# Symmetry related computation to get auxilary data and to test symmetrization features for MCMR 

In [3]:
import meshzoo
import numpy as np
import igl
import meshplot as mp
import scipy.io
import utils.geometry as gu
import meshplot_utils as mu

#to reload packages
import importlib
importlib.reload(mu)
importlib.reload(gu)
from matplotlib import cm 



In [4]:
shading_dic = dict(wireframe=True, width=1000)
resolutions= [2,3,4]
def reflectX(xyz):
    if len(xyz.shape)==2:
        return  np.concatenate( (-xyz[:,0:1], xyz[:,1:3]), axis=1) 
    else:
        return  np.concatenate( (-xyz[0:1], xyz[1:3]) ) 

npy_file     = '../auxilary/icoshpere_meshes.npy'
npy_sub_file = '../auxilary/icoshpere_mesh_subdivision_1.npy'


## Computing left-right partition of eco-sphere meshes wich were used to intinialize mean-shape in MCMR
* visualizing partition between vertices of left-right parts and vertices at the middle (x=0) and marking in red non-symmetric vertices (reported)
* Save these vertex partitions into [auxilary/icoshpere_partitions.npy](../auxilary/icoshpere_partitions.npy) 
* `V_left`, `V_right`, `V_middle` are vertex indixes of different parts and `V_opposite[i]` = index of the vertex opposite to the i_th vertex 

In [13]:
#save ico-sphere meshes and their partition 
p=[]
eps = 1e-6
icoshpere_meshes = dict()
for s_num in resolutions:
    V, F = meshzoo.icosa_sphere(n=s_num)

    V_right =  np.nonzero(V[:,0] > 0)[0]
    V_left  =  np.nonzero(V[:,0] < 0)[0]
    V_opposite = -np.ones((V.shape[0],),dtype=int)
    for v in range(0, V.shape[0]):
        #opposite_index = np.where(np.all(V == reflectX(V[v,:]), 1) )[0]
        opposite_index =  np.where(np.linalg.norm(V - reflectX(V[v,:]), axis =1) < eps)[0]
        if opposite_index.size:
            V_opposite[v] = opposite_index
        else:
            print(f'vertex {v}, coord  = {reflectX(V[v,:])}')
    V_middle = np.nonzero(np.arange(0,V_opposite.shape[0]) == V_opposite)[0]
    print(f'CheckSum: V_right + V_left + V_middle = {len(V_right) + len(V_left) + len(V_middle)}, Vert nium ={V.shape[0]}')
    # -W- set V_left = V_opposite[V_left] to macth right index order 
    icoshpere_meshes[s_num] = {'V':V,'F':F, 'V_left': np.nonzero(V[:,0] < 0)[0], 
                               'V_right':np.nonzero(V[:,0] > 0)[0], 
                               'V_middle':V_middle, 
                               'V_opposite':V_opposite,
                               }  
     
    slice_num = 3
    V_right_slice, V_left_slice = gu.add_horizontal_slices_to_symmetry_pro(icoshpere_meshes[s_num], V, slice_num)  ## computing left/right horizontal slices ('V_right_slice', 'V_left_slice')
    #V_right_slice, V_left_slice = gu.add_horizontal_slices_to_symmetry(icoshpere_meshes[s_num], V) ## computing left/right horizontal slices ('V_right_slice', 'V_left_slice')

    print(f'icosa_sphere{s_num}, ({V.shape[0]} vertices) non symmetric vertices = { np.where(V_opposite == -1)[0].tolist()}')
    print(f'X coordinate of middle vertices = {V[V_middle,0]}')

    color = np.zeros_like(V)
    color[V[:,0] < 0,:] = [0,1,0]
    color[V[:,0] > 0,:] = [0,0,1]
    color[V_opposite == -1,: ] = [1,0,0]
    print('Visualize Left-Right symmetry')
    if s_num == resolutions[0]: 
        V_mid_pnt = V[V_middle,:]
        p=mp.plot(V,F, shading = shading_dic)
        p.add_points(V, c =color, shading={"point_size": 1})
        p.add_points(V_mid_pnt, c = mu.color4plot([0.7,0.7,0.7], V_mid_pnt), shading={"point_size": 0.5})
    else:
        V_shifted = V+ [3*(s_num - resolutions[0]),0,0]
        V_mid_pnt = V_shifted[V_middle,:]
        p.add_mesh(V_shifted, F, shading = shading_dic)
        p.add_points(V_shifted, c =color, shading={"point_size": 1})
        p.add_points(V_mid_pnt, c = mu.color4plot([0.7,0.7,0.7], V_mid_pnt), shading={"point_size": 0.5})

    print(f'Visualize Left-Right symmetry with {slice_num} horizontal  slices from top to bottom ')
    color_slice = color.copy()
    vert_colors = cm.Set3(np.linspace(0, 1, slice_num))
    for si in range(slice_num):
        color_slice[V_right_slice[si],:] =  vert_colors[si,0:3]
        color_slice[V_left_slice[si], :] =  np.array([1,1,1]) - vert_colors[si,0:3]
    # color_slice[V_right_slice[1],:] =[1,1,0]
    # color_slice[V_left_slice[1], :] =[0,1,1]
    
    if s_num == resolutions[0]: 
        V_mid_pnt = V[V_middle,:]
        p1=mp.plot(V,F, shading = shading_dic)
        p1.add_points(V, c =color_slice, shading={"point_size": 1})
        p1.add_points(V_mid_pnt, c = mu.color4plot([0.7,0.7,0.7], V_mid_pnt), shading={"point_size": 0.5})

    else:
        V_shifted = V+ [3*(s_num - resolutions[0]),0,0]
        V_mid_pnt = V_shifted[V_middle,:]
        p1.add_mesh(V_shifted, F, shading = shading_dic)
        p1.add_points(V_shifted, c =color_slice, shading={"point_size": 1})
        p1.add_points(V_mid_pnt, c = mu.color4plot([0.7,0.7,0.7], V_mid_pnt), shading={"point_size": 0.5})


np.save(npy_file,icoshpere_meshes,  allow_pickle =True)

CheckSum: V_right + V_left + V_middle = 42, Vert nium =42
icosa_sphere2, (42 vertices) non symmetric vertices = []
X coordinate of middle vertices = [0. 0. 0. 0. 0. 0. 0. 0.]
Visualize Left-Right symmetry with 3 horizontal  slices from top to bottom 


Renderer(camera=PerspectiveCamera(aspect=1.6666666666666667, children=(DirectionalLight(color='white', intensi…

CheckSum: V_right + V_left + V_middle = 96, Vert nium =92
icosa_sphere3, (92 vertices) non symmetric vertices = []
X coordinate of middle vertices = [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -7.34506387e-17 -7.34506387e-17  7.34506387e-17  7.34506387e-17]
Visualize Left-Right symmetry with 3 horizontal  slices from top to bottom 
CheckSum: V_right + V_left + V_middle = 162, Vert nium =162
icosa_sphere4, (162 vertices) non symmetric vertices = []
X coordinate of middle vertices = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Visualize Left-Right symmetry with 3 horizontal  slices from top to bottom 


In [7]:
#visualize the saved data
data = np.load(npy_file, allow_pickle = True).item()
#print(data[2])


#p=[]
for s_num in  [2,3,4]:
    m = data[s_num]
    color = np.zeros_like(m['V'])
    color[m['V_left']] = [1,0,0] 
    color[m['V_right']] = [0,0,1] 
    V =m['V'].copy()
    
    V_right = m['V_right']
    V_opposite = m['V_opposite']
    #color4pairs =  V
    #color4pairs[V_right,:]  = V[V_opposite[V_right],:]
    color4pairs = np.array(range(0,V.shape[0]))
    color4pairs[V_right] = V_opposite[V_right]

    # if s_num ==2: 
    p=mp.plot(m['V'],m['F'], shading = shading_dic)
    p.add_points(m['V'], c =color, shading={"point_size": 0.8})
    p.add_points(m['V'], c =color4pairs, shading={"point_size": 0.5})
    # else:
    #     V_shifted = m['V']+ [3*(s_num - 2),0,0]
    #     p.add_mesh(V_shifted, m['F'], shading = shading_dic)
    #     p.add_points(V_shifted, c =color, shading={"point_size": 1})
    #     p.add_points(V_shifted, c =color4pairs, shading={"point_size": 0.5})
        


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Loading saved results to validate its correctness 
* First, visualzing vertex partition and coloring left-right vertex pairs by the same color
* Then, drawing meshes in which left vertices were reflected to the right 

In [8]:
# visualize the symmetrization 
p=[]
for s_num in resolutions:
    m = data[s_num]
    color = np.zeros_like(m['V'])
    color[m['V_left']]  = [1,0,0] 
    color[m['V_right']] = [0,0,1] 
    
    V = m['V'].copy() #reflect vertex coordinate from the right to the left
    #V[m['V_left'],:]  =  V[m['V_right'],:]
    #V[m['V_left'], 0] =  -V[m['V_left'], 0] 
    V_right = m['V_right']
    V_opposite = m['V_opposite']
    V[V_right,:] = V[V_opposite[V_right],:]
    V[V_right,0] = -V[V_opposite[V_right],0]

    if s_num ==2: 
        p = mp.plot(m['V'], m['F'], shading = shading_dic)
        p.add_points(V, c =color, shading={"point_size": 1})
    else:
        V_shifted = V + [3*(s_num - resolutions[0]),0,0]
        p.add_mesh(V_shifted, m['F'], shading = shading_dic)
        p.add_points(V_shifted, c =color, shading={"point_size": 1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Computing left-right partition of subdevided eco-sphere mesh which are used in MCMR evaluation/train
* visualizing partition between vertices of left-right parts and vertices at the middle (x=0) and marking in red non-symmetric vertices (reported)
* Save these vertex partitions into [auxilary/icoshpere_mesh_subdivision_1.npy](../auxilary/icoshpere_mesh_subdivision_1.npy) 
* `V_left`, `V_right`, `V_middle` are vertex indixes of different parts and `V_opposite[i]` = index of the vertex opposite to the i_th vertex icoshpere_mesh_subdivision_1

In [16]:
import torch 
from pytorch3d.ops import SubdivideMeshes
from pytorch3d.structures import Meshes


#save ico-sphere meshes and their partition 
p=[]
eps = 1e-6
icoshpere_meshes = dict()
shading_dic = dict(wireframe=True, width=1000)
resolutions= [2,3,4]
for s_num in resolutions:

    #subdivide eco-shpera using Pytorch3D fucnitos from MCMR model code 
    V_old, F_old = meshzoo.icosa_sphere(n=s_num) 
    old_mesh = Meshes(verts=[torch.from_numpy(V_old).detach()], faces=[torch.from_numpy(F_old).detach()])
    subdivider = SubdivideMeshes(old_mesh)
    new_mesh = subdivider(old_mesh)  # type: Meshes
    V = new_mesh.verts_packed().detach().cpu().numpy()
    F = new_mesh.faces_packed().detach().cpu().numpy()

    color = np.zeros_like(V)
    color[V[:,0] < 0,:] = [0,1,0]
    color[V[:,0] > 0,:] = [0,0,1]
    V_right =  np.nonzero(V[:,0] > 0)[0]
    V_left  =  np.nonzero(V[:,0] < 0)[0]
    V_opposite = -np.ones((V.shape[0],),dtype=int)
    for v in range(0, V.shape[0]):
        #opposite_index = np.where(np.all(V == reflectX(V[v,:]), 1) )[0]
        opposite_index =  np.where(np.linalg.norm(V - reflectX(V[v,:]), axis =1) < eps)[0]
        if opposite_index.size:
            V_opposite[v] = opposite_index
        else:
            print(f'vertex {v}, coord  = {reflectX(V[v,:])}')

    V_middle = np.nonzero(V[:,0] == 0)[0]
    print(f'CheckSum: V_right + V_left + V_middle = {len(V_right) + len(V_left) + len(V_middle)}, Vert nium ={V.shape[0]}')
    

    print(f'icosa_sphere{s_num}, ({V.shape[0]} vertices) non symmetric vertices = { np.where(V_opposite == -1)[0].tolist()}')
    color[V_opposite == -1,: ] = [1,0,0]
    # print('Visualize Left-Right symmetry')
    # if s_num == resolutions[0]: 
    #     V_mid_pnt = V[V_middle,:]
    #     p=mp.plot(V,F, shading = shading_dic)
    #     p.add_points(V, c =color, shading={"point_size": 1})
    #     p.add_points(V_mid_pnt, c = mu.color4plot([0.7,0.7,0.7], V_mid_pnt), shading={"point_size": 0.5})
    # else:
    #     V_shifted = V+ [3*(s_num - resolutions[0]),0,0]
    #     V_mid_pnt = V_shifted[V_middle,:]
    #     p.add_mesh(V_shifted, F, shading = shading_dic)
    #     p.add_points(V_shifted, c =color, shading={"point_size": 1})
    #     p.add_points(V_mid_pnt, c = mu.color4plot([0.7,0.7,0.7], V_mid_pnt), shading={"point_size": 0.5})
        
    
    icoshpere_meshes[s_num] = {'V':V,'F':F, 'V_left': np.nonzero(V[:,0] < 0)[0], 
                               'V_right':np.nonzero(V[:,0] > 0)[0], 
                               'V_middle':V_middle, 
                               'V_opposite':V_opposite}  
    slice_num = 3
    V_right_slice, V_left_slice = gu.add_horizontal_slices_to_symmetry_pro(icoshpere_meshes[s_num], V, slice_num)  ## computing left/right horizontal slices ('V_right_slice', 'V_left_slice')

    #V_right_slice, V_left_slice = gu.add_horizontal_slices_to_symmetry(icoshpere_meshes[s_num], V) ## computing left/right horizontal slices ('V_right_slice', 'V_left_slice')

    print('Visualize Left-Right symmetry with {slice_num} horizontal slices from top to bottom')
    color_slice = color.copy()
    vert_colors = cm.Set3(np.linspace(0, 1, slice_num))
    for si in range(slice_num):
        color_slice[V_right_slice[si],:] =  vert_colors[si,0:3]
        color_slice[V_left_slice[si], :] =  np.array([1,1,1]) - vert_colors[si,0:3]

    # color_slice[V_right_slice[1],:] =[1,1,0]
    # color_slice[V_left_slice[1], :] =[0,1,1]
    
    if s_num == resolutions[0]: 
        V_mid_pnt = V[V_middle,:]
        p1=mp.plot(V,F, shading = shading_dic)
        p1.add_points(V, c =color_slice, shading={"point_size": 1})
        p1.add_points(V_mid_pnt, c = mu.color4plot([0.7,0.7,0.7], V_mid_pnt), shading={"point_size": 0.5})

    else:
        V_shifted = V+ [3*(s_num - resolutions[0]),0,0]
        V_mid_pnt = V_shifted[V_middle,:]
        p1.add_mesh(V_shifted, F, shading = shading_dic)
        p1.add_points(V_shifted, c =color_slice, shading={"point_size": 1})
        p1.add_points(V_mid_pnt, c = mu.color4plot([0.7,0.7,0.7], V_mid_pnt), shading={"point_size": 0.5})

np.save(npy_sub_file,icoshpere_meshes,  allow_pickle =True)

CheckSum: V_right + V_left + V_middle = 162, Vert nium =162
icosa_sphere2, (162 vertices) non symmetric vertices = []
Visualize Left-Right symmetry with {slice_num} horizontal slices from top to bottom


Renderer(camera=PerspectiveCamera(aspect=1.6666666666666667, children=(DirectionalLight(color='white', intensi…

CheckSum: V_right + V_left + V_middle = 362, Vert nium =362
icosa_sphere3, (362 vertices) non symmetric vertices = []
Visualize Left-Right symmetry with {slice_num} horizontal slices from top to bottom
CheckSum: V_right + V_left + V_middle = 642, Vert nium =642
icosa_sphere4, (642 vertices) non symmetric vertices = []
Visualize Left-Right symmetry with {slice_num} horizontal slices from top to bottom


In [4]:
#visualize the saved data
data = np.load(npy_sub_file, allow_pickle = True).item()
for s_num in  [2,3,4]:
    m = data[s_num]
    color = np.zeros_like(m['V'])
    color[m['V_left']] = [1,0,0] 
    color[m['V_right']] = [0,0,1] 
    V =m['V'].copy()
    
    V_right = m['V_right']
    V_opposite = m['V_opposite']
    color4pairs = np.array(range(0,V.shape[0]))
    color4pairs[V_right] = V_opposite[V_right]

    p=mp.plot(m['V'],m['F'], shading = shading_dic)
    p.add_points(m['V'], c =color, shading={"point_size": 0.8})
    p.add_points(m['V'], c =color4pairs, shading={"point_size": 0.5})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Loading saved results to validate its correctness 
* First, visualzing vertex partition and coloring left-right vertex pairs by the same color
* Then, drawing meshes in which left vertices were reflected to the right 

In [8]:
# visualize the symmetrization 
p=[]
for s_num in resolutions:
    m = data[s_num]
    color = np.zeros_like(m['V'])
    color[m['V_left']]  = [1,0,0] 
    color[m['V_right']] = [0,0,1] 
    
    V = m['V'].copy() #reflect vertex coordinate from the right to the left
    #V[m['V_left'],:]  =  V[m['V_right'],:]
    #V[m['V_left'], 0] =  -V[m['V_left'], 0] 
    V_right = m['V_right']
    V_opposite = m['V_opposite']
    V[V_right,:] = V[V_opposite[V_right],:]
    V[V_right,0] = -V[V_opposite[V_right],0]

    if s_num ==2: 
        p = mp.plot(m['V'], m['F'], shading = shading_dic)
        p.add_points(V, c =color, shading={"point_size": 1})
    else:
        V_shifted = V + [3*(s_num - resolutions[0]),0,0]
        p.add_mesh(V_shifted, m['F'], shading = shading_dic)
        p.add_points(V_shifted, c =color, shading={"point_size": 1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…